In [68]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Pseudocode for cyclopeptide sequencing problem
    
    CyclopeptideSequencing(Spectrum)
        Peptides ← a set containing only the empty peptide
        while Peptides is nonempty
            Peptides ← Expand(Peptides)
            for each peptide Peptide in Peptides
                if Mass(Peptide) = ParentMass(Spectrum)
                    if Cyclospectrum(Peptide) = Spectrum
                        output Peptide
                    remove Peptide from Peptides
                else if Peptide is not consistent with Spectrum
                    remove Peptide from Peptides

In [69]:
# import integer mass table for amino acids

masses = {}
with open ("integer_mass_table.txt", "r") as myfile:
    for line in myfile:
        aa_mass = line.rstrip().split(" ")
        masses[aa_mass[0]] = int(aa_mass[1])

        
# extract just list of unique integer masses
mass_set = set(list(masses.values()))


In [70]:
# generate list of all sub-peptides from a circular sequence

def cyclic_subpeptides(pep_seq):
    n = len(pep_seq)
    cyclo_seq = pep_seq + pep_seq
    sub_peptides = []

    for size in range(1,n):
        for start in range(n):
            sub_peptides.append(cyclo_seq[start:start+size])

    sub_peptides.append(pep_seq)
    return(sub_peptides)


# generate list of all sub-peptides from a linear sequence

def linear_subpeptides(pep_seq):
    n = len(pep_seq)
    sub_peptides = []
    for size in range(1,n):
        for start in range(n-size+1):
            sub_peptides.append(pep_seq[start:start+size])

    sub_peptides.append(pep_seq)
    return(sub_peptides)


# calculate mass of each sub_peptide

def calc_mass(subpep_seq, mass_dict):
    mass = 0
    for i in subpep_seq:
        mass = mass + mass_dict[i]
    return(mass)


# calculate mass spectrum of circular peptide

def calc_spectrum(pep_seq, mass_dict):
    subpep_list = subpeptides(pep_seq)
    mass_list = [calc_mass(pep, mass_dict) for pep in subpep_list]
    mass_list.append(0)
    mass_list.sort()
    return(mass_list)

In [81]:

def cyclic_sequence(test_spec, masses):
    final_peptides = []
    parent_mass = max(test_spec)
    peptides = [[]]

    while len(peptides) != 0:
        new_peptides = []
        for pep in peptides:
            for mass in mass_set:

                new_pep = []
                new_pep.extend(pep)
                new_pep.append(mass)

                if sum(new_pep) == parent_mass:
                    new_pep_spec = sorted([sum(i) for i in cyclic_subpeptides(new_pep)])
                    if ((len(new_pep_spec) == len(test_spec)) and (all(i in test_spec for i in new_pep_spec))):
                        final_peptides.append(new_pep)
                        break

                consistent = True

                for sub in linear_subpeptides(new_pep):
                    sum_sub = sum(sub)
                    if sum_sub not in test_spec:
                        consistent = False
                        break

                if consistent:
                    new_peptides.append(new_pep)

        peptides = new_peptides

    return(final_peptides)

In [75]:
# test function on extra dataset
with open ("cyclic-spectrum.txt", "r") as myfile:
    for line in myfile:
        cyclic_spectrum = line.rstrip().split(" ")

cyclic_spectrum = [int(i) for i in cyclic_spectrum]
print(cyclic_spectrum)

[0, 71, 97, 99, 103, 113, 113, 114, 115, 131, 137, 196, 200, 202, 208, 214, 226, 227, 228, 240, 245, 299, 311, 311, 316, 327, 337, 339, 340, 341, 358, 408, 414, 424, 429, 436, 440, 442, 453, 455, 471, 507, 527, 537, 539, 542, 551, 554, 556, 566, 586, 622, 638, 640, 651, 653, 657, 664, 669, 679, 685, 735, 752, 753, 754, 756, 766, 777, 782, 782, 794, 848, 853, 865, 866, 867, 879, 885, 891, 893, 897, 956, 962, 978, 979, 980, 980, 990, 994, 996, 1022, 1093]


In [83]:
test_spectrum = sorted(cyclic_spectrum)[1:]
candidates = cyclic_sequence(test_spectrum, mass_set)

In [90]:
peps = []
for cand in candidates:
    peps.append("-".join([str(i) for i in cand]))

f = open("cyclic-sequence-example-results.txt", "w")
f.write(" ".join(peps))
f.close()

In [85]:
for cand in candidates:
    print(cand)

[97, 99, 115, 113, 113, 114, 131, 71, 137, 103]
[97, 103, 137, 71, 131, 114, 113, 113, 115, 99]
[99, 97, 103, 137, 71, 131, 114, 113, 113, 115]
[99, 115, 113, 113, 114, 131, 71, 137, 103, 97]
[131, 71, 137, 103, 97, 99, 115, 113, 113, 114]
[131, 114, 113, 113, 115, 99, 97, 103, 137, 71]
[71, 131, 114, 113, 113, 115, 99, 97, 103, 137]
[71, 137, 103, 97, 99, 115, 113, 113, 114, 131]
[103, 97, 99, 115, 113, 113, 114, 131, 71, 137]
[103, 137, 71, 131, 114, 113, 113, 115, 99, 97]
[137, 71, 131, 114, 113, 113, 115, 99, 97, 103]
[137, 103, 97, 99, 115, 113, 113, 114, 131, 71]
[113, 113, 114, 131, 71, 137, 103, 97, 99, 115]
[113, 113, 115, 99, 97, 103, 137, 71, 131, 114]
[113, 114, 131, 71, 137, 103, 97, 99, 115, 113]
[113, 115, 99, 97, 103, 137, 71, 131, 114, 113]
[114, 131, 71, 137, 103, 97, 99, 115, 113, 113]
[114, 113, 113, 115, 99, 97, 103, 137, 71, 131]
[115, 99, 97, 103, 137, 71, 131, 114, 113, 113]
[115, 113, 113, 114, 131, 71, 137, 103, 97, 99]


In [88]:
with open ("cyclic-sequence-example-answers.txt", "r") as myfile:
    for line in myfile:
        seq_example = line.rstrip().split(" ")
        
for ex in seq_example:
    print([int(i) for i in ex.split("-")])

[103, 137, 71, 131, 114, 113, 113, 115, 99, 97]
[103, 97, 99, 115, 113, 113, 114, 131, 71, 137]
[113, 113, 114, 131, 71, 137, 103, 97, 99, 115]
[113, 113, 115, 99, 97, 103, 137, 71, 131, 114]
[113, 114, 131, 71, 137, 103, 97, 99, 115, 113]
[113, 115, 99, 97, 103, 137, 71, 131, 114, 113]
[114, 113, 113, 115, 99, 97, 103, 137, 71, 131]
[114, 131, 71, 137, 103, 97, 99, 115, 113, 113]
[115, 113, 113, 114, 131, 71, 137, 103, 97, 99]
[115, 99, 97, 103, 137, 71, 131, 114, 113, 113]
[131, 114, 113, 113, 115, 99, 97, 103, 137, 71]
[131, 71, 137, 103, 97, 99, 115, 113, 113, 114]
[137, 103, 97, 99, 115, 113, 113, 114, 131, 71]
[137, 71, 131, 114, 113, 113, 115, 99, 97, 103]
[71, 131, 114, 113, 113, 115, 99, 97, 103, 137]
[71, 137, 103, 97, 99, 115, 113, 113, 114, 131]
[97, 103, 137, 71, 131, 114, 113, 113, 115, 99]
[97, 99, 115, 113, 113, 114, 131, 71, 137, 103]
[99, 115, 113, 113, 114, 131, 71, 137, 103, 97]
[99, 97, 103, 137, 71, 131, 114, 113, 113, 115]


In [91]:
# now the "real" test question
with open ("dataset_100_6.txt", "r") as myfile:
    for line in myfile:
        cyclic_spectrum = line.rstrip().split(" ")

cyclic_spectrum = [int(i) for i in cyclic_spectrum]
test_spectrum = sorted(cyclic_spectrum)[1:]
candidates = cyclic_sequence(test_spectrum, mass_set)

In [92]:
peps = []
for cand in candidates:
    peps.append("-".join([str(i) for i in cand]))

f = open("cyclic-sequence-results.txt", "w")
f.write(" ".join(peps))
f.close()